# Encontro 02

## Materias de apoio:
* The Scientist and Engineer's Guide to Digital Signal Processing, de By Steven W. Smith, Ph.D. (cápitulos 14 a 18)
  * Disponivel em http://www.dspguide.com
* Instrument Fundamentals: Complete Guide, White Paper da National Instruments (cápitulos "Acquiring an Analog Signal" e "Understanding FFTs and Windowing")
  * Disponivel em http://www.ni.com/white-paper/3214/en/

## Tópicos Abordados:
* Aplicação prática de FFT e seus problemas
  * Sinal de exemplo: tensão de uma rede elétrica residencial;
  * Características de um sinal digital amostrado: período de amostragem, efeito de quantização e ruído de medição;
  * Notação cartesiana e polar: conversão e usos;
  * Notação em decibéis: conversão e usos;
  * Janelamento e seus efeitos:
    * Retangular;
    * Hanning;
    * Hamming;
    * Blackman;
    * Flat-top.
* Filtros digitais e suas características:
  * Conceito de sistema, impulso e resposta a impulso;
  * Filtro ideal e função Sinc(x);
  * Convolução continua e discreta;
  * Tipos de filtros:
	* Filtro Passa-Baixas, Passa-Altas, Passa-Faixa e Rejeita-Faixa;
    * Filtros FIR (Finite Impulse Response) x IIR (Infinite Impulse Response);
    * Filtros Flats, do Tipo I e do Tipo II;
	* Otimizado para a resposta no tempo ou em frequência.

* Filtros de Resposta Finita ao Impulso (FIR):
  * Média Móvel (Moving Average);
  * Sinc-Janelado (Windowing-Sinc);
  * Filtro personalizado (Custom Filter).


In [1]:
# Importação de módulos e instalação de bibliotecas adicionais

# Importação do pacote matématico Math
import math

# Importação do pacote Pandas
import pandas as pd

# Importação dos pacotes NumPy e SciPy
import numpy as np
from scipy.interpolate import interp1d
from scipy import signal

# Importação do pacote PyPlot do MatPlotLib
import matplotlib.pyplot as plt

# Instalação e importação do pacote mpld3
!pip install mpld3
import mpld3

# importação do módulo timeit
import timeit

     |████████████████████████████████| 1.0MB 3.8MB/s 
  Created wheel for mpld3: filename=mpld3-0.5.1-cp36-none-any.whl size=364064 sha256=506d1f58f7ab725de6771c90eea8cd956556ec58f5294598bf6b7b48878d90d1
  Stored in directory: /root/.cache/pip/wheels/38/68/06/d119af6c3f9a2d1e123c1f72d276576b457131b3a7bf94e402
Successfully built mpld3


In [2]:
# Diretiva do Notebook para exibição de gráficos inline
%matplotlib inline

# Configuração do tamanho dos gráficos
plt.rcParams["figure.figsize"] = (20,10)

##Aplicação prática de FFT e seus problemas:

Quando se amostra digitalmente um sinal, várias distorções são introduzidas no sinal original. Algumas distorções importantes para se ter em mente são:

* Distorções causadas pelo período de amostragem. O período de amostragem define qual as frequências podem ser representadas e trabalhadas no sinal amostrado. Pelo teorema da amostragem, precisamos que a frequência de amostragem seja pelo menos duas vezes a frequência do maior sinal presente no sinal a ser amostrado. Uma frequência de amostragem por volta de cinco vezes a frequência máxima do sinal costuma a ter um bom desempenho. Essa margem de segurança permite reduzir a influência de filtros de anti-aliasing nas frequências desejadas, assim como mais espaço para trabalhar com as frequências do sistema. Frequências de amostragens muito grandes podem dificultar a análise de frequências baixas no sistema;

* Distorções causadas pela quantidade de amostras. A resolução de frequência de uma DFT está diretamente relacionada a quantidade de amostras usadas no cálculo. Por conta disso, maior quantidade de amostras permite uma melhor resolução e separação em frequência. No entanto, mais amostras também significa maior necessidade de memória e tempo de processamento. Uma quantidade muito pequena de amostras terá dificuldade de separar frequências próximas. É recomendável que a quantidade de amostras represente pelo menos algumas oscilações completas do sinal original;

* Distorções causadas por quantização do sinal: Quando o sinal é digitalizado (através de um ADC), ele deixa de ser analógico e se torna digital. Ou seja, ele passa de uma representação continua para uma representação quantizada (com uma quantidade fixa de possíveis valores). Por exemplo, um ADC de 12 bits permite 4096 diferentes representações. Nesse caso, o que acontece com um sinal que varia de 0 a 10.000 quando ele é quantizado em 4096 representações? Cada representação vai ser equivalente a quantidade de aproximadamente 2,442. Ou seja, quando o sinal original tiver valores com uma distância menor que 2,442, eles terão exatamente a mesma representação digital;

* Distorções causadas por arredondamento numérico: a representação binária usada em computadores faz com que valores na base 10 sejam convertidos para valores na base 2. Valores inteiros na base 10 conseguem ser perfeitamente representados na base 2, mas valores reais não conseguem. A representação de ponto-flutuante usada em computadores usualmente apresenta uma pequena diferença em relação ao valor esperado. Essas pequenas diferenças são cumulativas e o erro final depende da quantidade de operações executadas e da precisão numérica da representação;

* Distorções causadas por ruído de medição: ruído de medição aparece naturalmente em qualquer sistema de medição e pode ser causado por inúmeros fenômenos elétricos nos sensores, circuitos elétricos e até mesmo diretamente no ADC. Normalmente esse tipo de ruído é de baixa intensidade e de natureza aleatória, o que resulta no aparecimento de frequências altas no diagrama de frequências. Felizmente esses ruídos costumam a ter média nula, o que permite reduzi-los tirando a média dos sinais ou das DFTs ruidosas;

* Distorções causadas por janelamento: Coletar $N$ amostras de um sinal é equivalente a recortar um pedaço finito do sinal real. Ou seja, pegamos uma janela entre $T_1$ e $T_2$ de um sinal definido de ${-\infty}$ até ${+\infty}$. Se conseguirmos que essa janela seja uma representação perfeita das oscilações do sinal original (períodos completos de todas as frequências), teremos uma DFT sem nenhuma distorção. Na prática, dificilmente teremos essa situação. Usualmente teremos períodos incompletos, o que irá introduzir distorções de alta frequências no sinal. Podemos reduzir essas distorções através de diferentes janelamentos no sinal.



In [3]:
# Sinal de amostrado de exemplo: Tensão de uma Rede Elétrica Residencial.

# Definição das características do sinal
num_signal = 1024
num_samples = 256
measurement_noise = 0.01
quantization = 4096
quantization_limits = np.array([-200, 200])
sampling_rate_hz = 600

# Criação do vetor de tempo e vetores de dados
time_signal = np.linspace(0, num_samples/sampling_rate_hz, num_signal)
time_samples = np.linspace(0, num_samples/sampling_rate_hz, num_samples + 1)[:-1]
voltage_signal = 110*np.sqrt(2)*np.sin(2*math.pi*60*time_signal + math.pi/4) +  110/5*np.sqrt(2)*np.sin(2*math.pi*180*time_signal - math.pi/4)
voltage_samples = 110*np.sqrt(2)*np.sin(2*math.pi*60*time_samples + math.pi/4) +  110/5*np.sqrt(2)*np.sin(2*math.pi*180*time_samples - math.pi/4)
voltage_samples = voltage_samples + voltage_samples*np.random.rand(voltage_samples.size)*measurement_noise

# Efeito da quantização
quantized_samples = np.round(voltage_samples*quantization/(quantization_limits[1] - quantization_limits[0]))
quantized_voltage_samples = quantized_samples * (quantization_limits[1] - quantization_limits[0]) / quantization;

# Criação da figura
fig = plt.figure()
plt.plot(time_signal, voltage_signal, '-b')
plt.plot(time_samples, quantized_voltage_samples, ',r')
plt.title('Tensão da Rede Eletrica Residencial')
plt.ylabel('Amplitude [V]')
plt.xlabel('Tempo [s]')

# Criação e exibição de tooltips no gráfico
mpld3.plugins.connect(fig, mpld3.plugins.MousePosition(fontsize=14))
mpld3.display()


A decomposição em senos e cossenos que FFT realiza gera os valores em uma notação cartesiana e é muito interessante matematicamente, mas de difícil análise por humanos. Uma forma mais interessante de analisar os dados é através da notação polar, através de uma magnitude e fase.

A vantagem de usar a notação polar é a capacidade de associar de maneira direta uma magnitude (intensidade) e fase (atraso) a uma frequência específica. Essas informações costumam a ser mais interessantes do que a decomposição matemática do sinal. Por exemplo, uma frequência na notação cartesiana pode ter componentes tanto na parte real quanto na imaginaria. Nesse caso, como calcular a amplitude final daquela frequência? A notação polar resolve esse problema.

A conversão entre notação cartesiana e polar pode ser feita com as fórmulas:
> $M = Mag(A) = \sqrt{Re(A)^2 + Im(A)^2}$ \\
$\theta = Phase(A) = \arctan(\frac{Re(A)}{Im(A)})$

A conversão entre notação polar e cartesiana pode ser feita com as fórmulas:
> $a = Re(A) = M \cos(\theta)$ \\
$b = Im(A) = M \sin(\theta)$

Ou seja:
> $a + jb = M (\cos(\theta) + j \sin(\theta))$


In [4]:
# Aplicação da FFT no sinal

# Execução do algoritmo da FFT
fft_ex = np.fft.fft(quantized_voltage_samples, num_samples)

# Geração do vetor de frequências da FFT
freqs_ex = np.fft.fftfreq(num_samples)

# Shift da FFT e do vetor de frequências
fft_ex_shifted = np.fft.fftshift(fft_ex)
freqs_ex_shifted = np.fft.fftshift(freqs_ex)

# Corrige o valor da FFT
fft_ex_scaled = fft_ex_shifted/num_samples
freqs_ex_shifted_hz = freqs_ex_shifted * sampling_rate_hz

# Seleciona a FFT a ser plotada
#fft_ex_plot = fft_ex
#fft_ex_plot = fft_ex_shifted
fft_ex_plot = fft_ex_scaled

# Seleciona o vetor de frequências ser plotado
#freqs_ex_plot = freqs_ex
#freqs_ex_plot = freqs_ex_shifted
freqs_ex_plot = freqs_ex_shifted_hz

# Criação da figura
fig = plt.figure()

# Criação do primeiro subplot
plt.subplot(2, 1, 1)
stem_fft_ex_real = plt.stem(freqs_ex_plot, np.real(fft_ex_plot), use_line_collection=True)
stem_fft_ex_real[0].set_markerfacecolor('none')
stem_fft_ex_real[0].set_markersize(2)
plt.title('Parte Real da FFT da Tensão Elétrica Residencial')
plt.ylabel('Amplitude [V]')
plt.xlabel('Frequência [Hz]')

# Criação do segundo subplot
plt.subplot(2, 1, 2)
stem_fft_ex_imag = plt.stem(freqs_ex_plot, np.imag(fft_ex_plot), use_line_collection=True)
stem_fft_ex_imag[0].set_markerfacecolor('none')
stem_fft_ex_imag[0].set_markersize(2)
plt.title('Parte Imaginária da FFT da Tensão Elétrica Residencial')
plt.ylabel('Amplitude [V]')
plt.xlabel('Frequência [Hz]')

# Criação e exibição de tooltips no gráfico
mpld3.plugins.connect(fig, mpld3.plugins.MousePosition(fontsize=14))
labels_fft_ex_real = ["Freq {:.3f} = {:.3f}".format(freqs_ex_plot[i], np.real(fft_ex_plot[i])) for i in range(freqs_ex_plot.size)]
mpld3.plugins.connect(fig, mpld3.plugins.PointLabelTooltip(stem_fft_ex_real[0], labels_fft_ex_real))
labels_fft_ex_imag = ["Freq {:.3f} = {:.3f}".format(freqs_ex_plot[i], np.imag(fft_ex_plot[i])) for i in range(freqs_ex_plot.size)]
mpld3.plugins.connect(fig, mpld3.plugins.PointLabelTooltip(stem_fft_ex_imag[0], labels_fft_ex_imag))
mpld3.display()


In [ ]:
# Exibição da FFT na forma polar

# Criação da figura
fig = plt.figure()

# Criação do primeiro subplot
plt.subplot(2, 1, 1)
stem_fft_ex_real = plt.stem(freqs_ex_shifted_hz, np.absolute(fft_ex_plot), use_line_collection=True)
stem_fft_ex_real[0].set_markerfacecolor('none')
stem_fft_ex_real[0].set_markersize(2)
plt.title('Magnitudes da FFT da Tensão Elétrica Residencial')
plt.ylabel('Amplitude [V]')
plt.xlabel('Frequência [Hz]')

# Criação do segundo subplot
plt.subplot(2, 1, 2)
stem_fft_ex_imag = plt.stem(freqs_ex_shifted_hz, np.angle(fft_ex_plot), use_line_collection=True)
stem_fft_ex_imag[0].set_markerfacecolor('none')
stem_fft_ex_imag[0].set_markersize(2)
plt.title('Fases da FFT da Tensão Elétrica Residencial')
plt.ylabel('Fase [rad]')
plt.xlabel('Frequência [Hz]')

# Criação e exibição de tooltips no gráfico
mpld3.plugins.connect(fig, mpld3.plugins.MousePosition(fontsize=14))
labels_fft_ex_real = ["Freq {:.3f} = {:.3f}".format(freqs_ex_shifted_hz[i], np.absolute(fft_ex_plot[i])) for i in range(freqs_ex_shifted_hz.size)]
mpld3.plugins.connect(fig, mpld3.plugins.PointLabelTooltip(stem_fft_ex_real[0], labels_fft_ex_real))
labels_fft_ex_imag = ["Freq {:.3f} = {:.3f}".format(freqs_ex_shifted_hz[i], np.angle(fft_ex_plot[i])) for i in range(freqs_ex_shifted_hz.size)]
mpld3.plugins.connect(fig, mpld3.plugins.PointLabelTooltip(stem_fft_ex_imag[0], labels_fft_ex_imag))
mpld3.display()


Umas das razões para usar a notação polar é facilitar a análise dos ganhos em frequência de um sistema. Isso é, a relação entre a amplitude de um sinal antes e depois de um determinado sistema ou operação. Quando plotamos esses ganhos, podemos fazê-los em escala linear ou em escala logarítmica. A escala linear permite analisar melhor pequenas variações no sinal enquanto a escala logarítmica permite analisar melhor grandes variações. 
Usualmente é utilizado o decibel para representar a escala logarítmica. Um bel (em homenagem a Alexander Graham Bell) significa que a potência de um sistema é modificada por um fator de dez vezes. Um decibel (dB) é um decimo de um bel. Por exemplo, os valores em decibel -20dB, -10dB, 0dB, 10dB, 20dB significam que a potência é modificada pelos fatores 0,01, 0,1, 1, 10 e 100, respectivamente.
O decibel (dB) representa uma razão entre duas potências, o que significa que ela depende de uma potência de referência para ter significado. Podemos matematicamente definir o decibel como:
> $dB = 10\log_{10}(\frac{P_2}{P_1})$

Temos um problema nessa representação: ela depende da potência de um sinal, mas usualmente trabalhamos com amplitudes de sinais. A amplitude de um sinal é proporcional a raiz-quadrada de sua potência. O que significa que uma amplificação de 20dB representa uma razão de 100 vezes na potência, mas apenas uma razão de 10 na amplitude. Ou seja, a cada 20 decibéis o valor da amplitude é modificado em dez vezes. Matematicamente:
> $dB = 20\log_{10}(\frac{A_2}{A_1})$

Também podemos usar como referência valores pré-determinados, como no dBV (referenciado a um sinal de 1 volt rms) ou no dDm (referenciado a um sinal produzindo 1 mW em uma carga de 600 ohm).

Duas coisas importantes para se lembrar do uso de decibéis em sistemas são:
* Um ganho de -3dB significa uma redução de raiz2 na amplitude de 2 e na potência.
* A cada 20db você tem um ganho de 10 vezes na amplitude do sinal e 100 vezes na potência.


In [6]:
# Exibição da FFT na forma polar, com a magnetude em decibeis

# Criação da figura
fig = plt.figure()

# Criação do primeiro subplot
plt.subplot(2, 1, 1)
stem_fft_ex_real = plt.stem(freqs_ex_shifted_hz, 20*np.log10(np.absolute(fft_ex_plot)), use_line_collection=True)
stem_fft_ex_real[0].set_markerfacecolor('none')
stem_fft_ex_real[0].set_markersize(2)
plt.title('Magnitudes da FFT da Tensão Elétrica Residencial')
plt.ylabel('Amplitude [dB]')
plt.xlabel('Frequência [Hz]')

# Criação do segundo subplot
plt.subplot(2, 1, 2)
stem_fft_ex_imag = plt.stem(freqs_ex_shifted_hz, np.angle(fft_ex_plot), use_line_collection=True)
stem_fft_ex_imag[0].set_markerfacecolor('none')
stem_fft_ex_imag[0].set_markersize(2)
plt.title('Fases da FFT da Tensão Elétrica Residencial')
plt.ylabel('Fase [rad]')
plt.xlabel('Frequência [Hz]')

# Criação e exibição de tooltips no gráfico
mpld3.plugins.connect(fig, mpld3.plugins.MousePosition(fontsize=14))
labels_fft_ex_real = ["Freq {:.3f} = {:.3f}".format(freqs_ex_shifted_hz[i],  20*np.log10(np.absolute(fft_ex_plot[i]))) for i in range(freqs_ex_shifted_hz.size)]
mpld3.plugins.connect(fig, mpld3.plugins.PointLabelTooltip(stem_fft_ex_real[0], labels_fft_ex_real))
labels_fft_ex_imag = ["Freq {:.3f} = {:.3f}".format(freqs_ex_shifted_hz[i], np.angle(fft_ex_plot[i])) for i in range(freqs_ex_shifted_hz.size)]
mpld3.plugins.connect(fig, mpld3.plugins.PointLabelTooltip(stem_fft_ex_imag[0], labels_fft_ex_imag))
mpld3.display()


Uma das características da DFT é que ela assume que o sinal discreto a ser transformado é periódico, e definido $-\infty{}$ até o $+\infty{}$. Como precisamos coletar um número finito de amostras para representar digitalmente o sinal realizar o algoritmo computacionalmente, temos uma sub-representação do sinal original. Para compensar esse fato, o algoritmo da DFT assume que as amostras fornecidas são uma quantidade inteira de ciclos desse sinal periódico. Ou seja, se repetimos infinitamente as amostras coletadas (conectando a última amostra a primeira amostra), teremos o sinal periódico original.

Essas amostras do sinal original são uma janela finita do sinal original. O que acontece com a DFT se essa janela não conseguir representar uma quantidade inteira de ciclos do sinal periódico? Temos uma deformação no sinal quando conectamos a última com a primeira amostra. Essa deformação introduz frequências altas no espectro de frequências, assim como erros nas amplitudes das frequências. Uma maneira de minimizar esse efeito é reduzir as amplitudes das extremidades do sinal amostrado, de forma a minimizar as variações quando conectamos a última e a primeira a amostra do sinal.

Simplesmente recortar um pedaço do sinal original é o equivalente a usar uma janela retangular, que aplica um ganho unitário em todos os pontos da amostra. Ou seja, uma janela é definida pelos ganhos a serem aplicados a cada amostra do sinal. Em especial, queremos janelas com ganhos pequenos nas extremidades e ganhos unitários nas amostras centrais. Iremos abordar algumas das janelas mais comumente usadas: Retangular, Hanning, Hamming, Blackman e Flat Top.

Podemos definir as características de uma janela através de sua resposta em frequência, particularmente em função do seu lóbulo principal e seus lóbulos secundários. O lóbulo principal é centrado em cada componente de frequência e os lóbulos secundários se aproximam a zero. O efeito de aplicar uma janela é definido por esses dois lóbulos:
* Um lóbulo principal mais estreito permite uma melhor separação de frequências, mas implica em ter lóbulos secundários com maiores ganhos;
* Lóbulos secundários com ganho menores e taxas de decaimento mais rápidas reduzem o espraiamento espectral, mas implicam em um lóbulo principal mais largo (com maior largura de banda).

A escolha da janela depende das características de frequência do sinal, visto que diferentes janelas possuem diferentes características e aplicações. Algumas regras práticas a serem consideradas ao se escolher uma janela são:
* Se o sinal contém frequências interferentes fortes e distantes da frequência de interesse, escolha uma janela suavizadora com alta taxa de decaimento nos lóbulos secundários;
* Se o sinal contém frequências interferentes fortes e próximas da frequência de interesse, escolha uma janela com um ganho máximo pequeno para os lóbulos secundários;
* Se a frequência de interesse contém dois ou mais sinais muito próximos um do outro, resolução espectral é importante. Nesse caso, é melhor escolher uma janela suavizadora com um lóbulo principal bem estreito;
* Se a acurácia de amplitude de um único componente de frequência é mais importante que a sua localização exata da componente em uma dada raia de frequência, escolha uma janela com um lóbulo principal largo;
* Se o espectro de um sinal é bastante plano ou com amplo conteúdo de frequência, use a janela retangular (também conhecida como uniforme);
* Em geral, a janela de Hanning (Hann) é satisfatória em 95% dos casos. Ela tem uma boa resolução de frequência e espraiamento espectral reduzido. Se a natureza do sinal não é conhecido e deseja-se aplicar uma janela suavizadora, comece com uma janela de Hann.

Pode ser necessário comparar diferentes janelas e seus efeitos em um determinado sinal para localizar a janela mais apropriada para um determinado problema. Sugestões de janelas em razão do conteúdo de sinal sob análise podem ser encontradas na tabela abaixo:

| Conteúdo do sinal                                                   | Janela              |
|---------------------------------------------------------------------|---------------------|
| Senoidal ou combinação de senoides                                  | Hann                |
| Senoidal (acurácia de amplitude é importante)                       | Flat Top            |
| Aleatório de banda estreita (dados de vibração)                     | Hann                |
| Aleatório de banda larga (ruído branco)                             | Retangular          |
| Senoides muito próximas                                             | Retangular, Hamming |
| Sinais de excitação (golpe de um martelo)                           | Force               |
| Sinais de resposta                                                  | Exponencial         |
| Conteúdo desconhecido                                               | Hann                |
| Dois tons com frequências próximas, mas amplitudes muito diferentes | Kaiser-Bessel       |
| Dois tons com frequências próximas e amplitudes quase iguais        | Retangular          |
| Medição precisa da amplitude um único tom                           | Flat Top            |

A lista e documetnação das funções de janelamento implementadas em Python, no pacote SciPy, podem ser encontradas em: https://docs.scipy.org/doc/scipy/reference/signal.windows.html#module-scipy.signal.windows

In [8]:
# Resposta no tempo e em frequência da janela retangular

# Geração das janelas
num_window = 51
window_uniform = np.ones(num_window)

# Execução do algoritmo da FFT da janela Retangular
fft_uniform = np.fft.fft(window_uniform, 2048) / (len(window_uniform)/2.0)
freq_uniform = np.linspace(-0.5, 0.5, len(fft_uniform))
response_uniform = np.abs(np.fft.fftshift(fft_uniform / abs(fft_uniform).max()))
response_uniform = 20 * np.log10(np.maximum(response_uniform, 1e-10))

# Criação da figura
fig = plt.figure()

# Criação do primeiro subplot
plt.subplot(2, 1, 1)
plt.plot(window_uniform)
plt.title("Janela Retangular")
plt.ylabel("Amplitude")
plt.xlabel("Amostra")

# Criação do segundo subplot
plt.subplot(2, 1, 2)
plt.plot(freq_uniform, response_uniform)
plt.axis([-0.5, 0.5, -120, 0])
plt.title("Resposta em Frequência da Janela Retangular")
plt.ylabel("Magnitude Normalizada [dB]")
plt.xlabel("Frequência Normalizada [ciclos por amostra]")

# Criação e exibição de tooltips no gráfico
mpld3.plugins.connect(fig, mpld3.plugins.MousePosition(fontsize=14))
mpld3.display()


In [13]:
# Resposta no tempo e em frequência da janela de Hanning (Hann) e Hamming

# Geração das janelas
num_window = 51
window_hann = signal.hann(num_window)
window_hamming = signal.hamming(num_window)

# Execução do algoritmo da FFT da janela Hanning (Hann)
fft_hann = np.fft.fft(window_hann, 2048) / (len(window_hann)/2.0)
freq_hann = np.linspace(-0.5, 0.5, len(fft_hann))
response_hann = np.abs(np.fft.fftshift(fft_hann / abs(fft_hann).max()))
response_hann = 20 * np.log10(np.maximum(response_hann, 1e-10))

# Execução do algoritmo da FFT da janela Hamming
fft_hamming = np.fft.fft(window_hamming, 2048) / (len(window_hamming)/2.0)
freq_hamming = np.linspace(-0.5, 0.5, len(fft_hamming))
response_hamming = np.abs(np.fft.fftshift(fft_hamming / abs(fft_hamming).max()))
response_hamming = 20 * np.log10(np.maximum(response_hamming, 1e-10))

# Criação da figura
fig = plt.figure()

# Criação do primeiro subplot
plt.subplot(2, 2, 1)
plt.plot(window_hann)
plt.title("Janela de Hanning (Hann)")
plt.ylabel("Amplitude")
plt.xlabel("Amostra")

# Criação do segundo subplot
plt.subplot(2, 2, 2)
plt.plot(window_hamming)
plt.title("Janela de Hamming")
plt.ylabel("Amplitude")
plt.xlabel("Amostra")

# Criação do terceiro subplot
plt.subplot(2, 2, 3)
plt.plot(freq_hann, response_hann)
plt.axis([-0.5, 0.5, -120, 0])
plt.title("Resposta em Frequência da Janela de Hanning (Hann)")
plt.ylabel("Magnitude Normalizada [dB]")
plt.xlabel("Frequência Normalizada [ciclos por amostra]")

# Criação do quarto subplot
plt.subplot(2, 2, 4)
plt.plot(freq_hamming, response_hamming)
plt.axis([-0.5, 0.5, -120, 0])
plt.title("Resposta em Frequência da Janela de Hamming")
plt.ylabel("Magnitude Normalizada [dB]")
plt.xlabel("Frequência Normalizada [ciclos por amostra]")

# Criação e exibição de tooltips no gráfico
mpld3.plugins.connect(fig, mpld3.plugins.MousePosition(fontsize=14))
mpld3.display()


In [16]:
# Resposta no tempo e em frequência da janela de Blackman e Flat Top

# Geração das janelas
num_window = 51
window_blackman = signal.blackman(num_window)
window_flattop = signal.flattop(num_window)

# Execução do algoritmo da FFT da janela Blackman
fft_blackman = np.fft.fft(window_blackman, 2048) / (len(window_blackman)/2.0)
freq_blackman = np.linspace(-0.5, 0.5, len(fft_blackman))
response_blackman = np.abs(np.fft.fftshift(fft_blackman / abs(fft_blackman).max()))
response_blackman = 20 * np.log10(np.maximum(response_blackman, 1e-10))

# Execução do algoritmo da FFT da janela Flat Top
fft_flattop = np.fft.fft(window_flattop, 2048) / (len(window_flattop)/2.0)
freq_flattop = np.linspace(-0.5, 0.5, len(fft_flattop))
response_flattop = np.abs(np.fft.fftshift(fft_flattop / abs(fft_flattop).max()))
response_flattop = 20 * np.log10(np.maximum(response_flattop, 1e-10))

# Criação da figura
fig = plt.figure()

# Criação do primeiro subplot
plt.subplot(2, 2, 1)
plt.plot(window_blackman)
plt.title("Janela de Blackman")
plt.ylabel("Amplitude")
plt.xlabel("Amostra")

# Criação do segundo subplot
plt.subplot(2, 2, 2)
plt.plot(window_flattop)
plt.title("Janela Flat Top")
plt.ylabel("Amplitude")
plt.xlabel("Amostra")

# Criação do terceiro subplot
plt.subplot(2, 2, 3)
plt.plot(freq_blackman, response_blackman)
plt.axis([-0.5, 0.5, -120, 0])
plt.title("Resposta em Frequência da Janela de Blackman")
plt.ylabel("Magnitude Normalizada [dB]")
plt.xlabel("Frequência Normalizada [ciclos por amostra]")

# Criação do quarto subplot
plt.subplot(2, 2, 4)
plt.plot(freq_flattop, response_flattop)
plt.axis([-0.5, 0.5, -120, 0])
plt.title("Resposta em Frequência da Janela Flat Top")
plt.ylabel("Magnitude Normalizada [dB]")
plt.xlabel("Frequência Normalizada [ciclos por amostra]")

# Criação e exibição de tooltips no gráfico
mpld3.plugins.connect(fig, mpld3.plugins.MousePosition(fontsize=14))
mpld3.display()


In [17]:
# Aplicação de janelas na Tensão de uma Rede Elétrica Residencial

# Aplicação da janela
quantized_voltage_samples_windowed = quantized_voltage_samples * signal.hann(quantized_voltage_samples.size)
#quantized_voltage_samples_windowed = quantized_voltage_samples * signal.hamming(quantized_voltage_samples.size)
#quantized_voltage_samples_windowed = quantized_voltage_samples * signal.blackman(quantized_voltage_samples.size)
#quantized_voltage_samples_windowed = quantized_voltage_samples * signal.flattop(quantized_voltage_samples.size)

voltage_signal_windowed = voltage_signal * signal.hann(voltage_signal.size)
#voltage_signal_windowed = voltage_signal * signal.hamming(voltage_signal.size)
#voltage_signal_windowed = voltage_signal * signal.blackman(voltage_signal.size)
#voltage_signal_windowed = voltage_signal * signal.flattop(voltage_signal.size)

# Criação da figura
fig = plt.figure()
plt.plot(time_signal, voltage_signal_windowed, '-b')
plt.plot(time_samples, quantized_voltage_samples_windowed, ',r')
plt.title('Tensão da Rede Eletrica Residencial, janelada')
plt.ylabel('Amplitude [V]')
plt.xlabel('Tempo [s]')

# Criação e exibição de tooltips no gráfico
mpld3.plugins.connect(fig, mpld3.plugins.MousePosition(fontsize=14))
mpld3.display()

In [18]:
# Aplicação da FFT janelada na forma polar

# Execução do algoritmo da FFT
fft_windowed = np.fft.fft(quantized_voltage_samples_windowed, num_samples)

# Geração do vetor de frequências da FFT
freqs_windowed = np.fft.fftfreq(num_samples)

# Shift da FFT e do vetor de frequências
fft_windowed_shifted = np.fft.fftshift(fft_windowed)
freqs_windowed_shifted = np.fft.fftshift(freqs_windowed)

# Corrige o valor da FFT
fft_windowed_scaled = fft_windowed_shifted/num_samples
freqs_windowed_shifted_hz = freqs_windowed_shifted * sampling_rate_hz

# Seleciona a FFT a ser plotada
#fft_windowed_plot = fft_windowed
#fft_windowed_plot = fft_windowed_shifted
fft_windowed_plot = fft_windowed_scaled

# Seleciona o vetor de frequências ser plotado
#freqs_windowed_plot = freqs_windowed
#freqs_windowed_plot = freqs_windowed_shifted
freqs_windowed_plot = freqs_windowed_shifted_hz

# Criação da figura
fig = plt.figure()

# Criação do primeiro subplot
plt.subplot(2, 1, 1)
stem_fft_windowed_real = plt.stem(freqs_windowed_shifted_hz, np.absolute(fft_windowed_plot), use_line_collection=True)
stem_fft_windowed_real[0].set_markerfacecolor('none')
stem_fft_windowed_real[0].set_markersize(2)
plt.title('Magnitudes da FFT da Tensão Elétrica Residencial Janelada')
plt.ylabel('Amplitude [V]')
plt.xlabel('Frequência [Hz]')

# Criação do segundo subplot
plt.subplot(2, 1, 2)
stem_fft_windowed_imag = plt.stem(freqs_windowed_shifted_hz, np.angle(fft_windowed_plot), use_line_collection=True)
stem_fft_windowed_imag[0].set_markerfacecolor('none')
stem_fft_windowed_imag[0].set_markersize(2)
plt.title('Fases da FFT da Tensão Elétrica Residencial Janelada')
plt.ylabel('Fase [rad]')
plt.xlabel('Frequência [Hz]')

# Criação e exibição de tooltips no gráfico
mpld3.plugins.connect(fig, mpld3.plugins.MousePosition(fontsize=14))
labels_fft_windowed_real = ["Freq {:.3f} = {:.3f}".format(freqs_windowed_shifted_hz[i], np.absolute(fft_windowed_plot[i])) for i in range(freqs_windowed_shifted_hz.size)]
mpld3.plugins.connect(fig, mpld3.plugins.PointLabelTooltip(stem_fft_windowed_real[0], labels_fft_windowed_real))
labels_fft_windowed_imag = ["Freq {:.3f} = {:.3f}".format(freqs_windowed_shifted_hz[i], np.angle(fft_windowed_plot[i])) for i in range(freqs_windowed_shifted_hz.size)]
mpld3.plugins.connect(fig, mpld3.plugins.PointLabelTooltip(stem_fft_windowed_imag[0], labels_fft_windowed_imag))
mpld3.display()


O próximo passo após converter um sinal do tempo para a frequência é analisar e manipular as frequências do sinal. A ferramenta que nos permite fazer isso são os filtros. 

Filtros podem ser analógicos ou digitais e tem como principal função filtrar (remover) determinadas frequências de um sinal. No entanto, filtros podem ser considerados elementos que manipulam no tempo a frequência de um sinal, seja para atenuar ou amplificar diferentes frequências. 

Podemos pensar em filtros como um sistema, que recebe um sinal de entrada e entrega um sinal de saída modificado. Diferente de um sinal, um sistema não tem uma representação no tempo ou em frequência. Nesse caso, como podemos caracterizar diferentes sistemas? Uma maneira é utilizar sua resposta a um sinal conhecido. Conhecendo qual sinal estimulou o sistema e a saída do sistema a esse sinal, podemos descrever completamente esse sistema. Para esse fim utilizado um sinal chamado impulso, e a resposta do sistema a esse sinal de resposta ao impulso (ou resposta impulsiva).

A função Delta de Dirac, $\delta(x)$, também conhecida como função impulso, é definida como uma função com todos os valores zerados exceto por um impulso de amplitude infinita e área unitária no momento $x$. Matematicamente:
> $\delta(x-x_0) = \left\{ \begin{matrix}{} \infty{}, & x=x_0 \\ 0, & x \neq x_0 \end{matrix} \right.$ \\
$\int_{-\infty}^{\infty}\delta(x) dx = 1$

Porque a função $\delta(x)$ é interessante para a parametrização de um sistema? Porque o $\delta(x)$ representa um ponto infinitesimal no tempo e uma linha continua no domínio da frequência, com ganho unitário. Ou seja, ao excitar um sistema por um impulso, estamos excitando igualmente todas as frequências possíveis, o que permite ter uma representação fiel da resposta em frequência do sistema. A resposta impulsiva de um sistema no domínio da frequência representa o que acontece com cada frequência do sinal original quando passa pelo sistema. Se colocado em coordenadas polares, representa o ganho de amplitude e a mudança de fase que cada frequência sofre.

Digitalmente não podemos representar um sinal infinito com área unitária. Quando escrito na forma discreta, $\delta[n]$, podemos escrever a função impulso como:
> $\delta[n-n_0] = \left\{ \begin{matrix}{} 1, & n=n_0 \\ 0, & n \neq n_0 \end{matrix} \right.$ \\
$\sum_{n=0}^{N-1}\delta[n] = 1$

A função $\delta[n]$ é de grande interesse por parte de sistemas discretos, pois podemos representar um vetor de amostras como um trem de impulso com pesos, sendo um impulso por amostra. Como estamos trabalhando com sistemas lineares, a resposta de um sistema a um conjunto de amostras é equivalente a somatória das respostas impulsivas desse sistema a cada amostra individual. Essa propriedade é o que permite a execução da convolução no tempo, permitindo a aplicação de filtros no tempo.

Convolução continua é uma operação matemática definida por:
> $Conv(f,g) = (f * g)(t) = h(t) = \int_{-\infty}^{+\infty} f(\tau)g(t-\tau)d\tau$

Enquanto a convolução discreta é definida por:
> $Conv(f,g) = (f * g)[k] = h[k] = \sum_{j=0}^{k} f[j]g[k-j]$

Aonde $f$ e $g$ são sinais com $n$ amostras, e o resultado da convolução delas possui o tamanho $2n-1$. Isso é, $k$ assume valores entre $[0, 2n-2]$.

Pelas equações das convoluções, podemos perceber que a convolução de um sinal qualquer com um impulso resulta no mesmo sinal, com atraso equivalente ao do impulso. Ou seja, a resposta impulsiva de um sistema é equivalente a resposta “real” do sistema.

A razão da convolução ser tão importante para PDS é que ela é a operação dual da multiplicação para os reinos do tempo e frequência. Ou seja: 
* Uma operação de multiplicação no tempo equivale a uma operação de convolução na frequência;
* Uma operação de convolução no tempo equivale a uma operação de multiplicação na frequência.

Ou seja, é possível aplicar ganhos individuais nas frequências de um sinal aplicando a convolução desse sinal no tempo com um filtro.

Lembrando que o espectro de frequência da resposta impulsiva de um filtro define os ganhos em frequência desse filtro, queremos multiplicar o diagrama de frequências do nosso sinal de interesse com o do filtro. Podemos fazer exatamente essa mesma operação aplicando a convolução do sinal de interesse com a resposta impulsiva do filtro, tudo no domínio do tempo.



##Filtros digitais e suas características:

Podemos então, enfim, definir o que é um filtro: um sistema com resposta impulsiva que tenha por efeito eliminar certas frequências (ganho nulo) e manter outras (ganho unitário). Esse seria um filtro ideal.

Por exemplo, um o filtro passa-baixas ideal tem sua resposta em frequência no formato de um pulso unitário, com resposta:
> $H(\omega) = \left\{ \begin{matrix}{} 0, & -0,5 \leq \omega \geq -\omega_c \\ 1, & -\omega_c < \omega > \omega_c \\ 0, & \omega_c \leq \omega \geq 0,5 \end{matrix} \right.$ \\

Quando convertemos esse sinal para o domínio do tempo, obtemos a resposta impulsiva definida pela função sinc normalizada:
> $sinc(x) = \frac{\sin(\pi x)}{\pi x}$

Logo, em teoria, podemos gerar um filtro passa baixas perfeito usando a função $sinc(x)$. No entanto, essa função é infinita, não condizente com sistemas amostrados. Portanto, embora seja possível gerar filtros baseados na função $sinc(x)$, é impossível em um sistema amostrado gerar um filtro perfeito (embora seja possível conseguir respostas próximas).


In [19]:
# Filtro ideal e função sinc(x)

# Definição do filtro ideal ma DFT
num_pontos = 512
dft_filtro_ideal = np.append(np.append(np.zeros(np.int(num_pontos/4)), np.ones(np.int(num_pontos/2))),np.zeros(np.int(num_pontos/4)))
freqs_filtro_ideal = np.fft.fftshift(np.fft.fftfreq(num_pontos))

# Aplicação da IFFT na resposta em frequência
sinal_filtro_ideal = np.real(np.fft.fftshift(np.fft.ifft(np.fft.fftshift(dft_filtro_ideal), num_pontos))) * 2

# Criação da figura
fig = plt.figure()

# Criação do primeiro subplot
plt.subplot(2, 1, 1)
plt.plot(freqs_filtro_ideal, dft_filtro_ideal)
plt.title('Resposta em Frequência de um filtro passa-baixas ideal')
plt.ylabel('Ganho')
plt.xlabel('Frequência [ciclos por amostra]')

# Criação do segundo subplot
plt.subplot(2, 1, 2)
plt.plot(sinal_filtro_ideal)
plt.title('Resposta no Tempo de um filtro passa-baixas idea')
plt.ylabel('Amplitude')
plt.xlabel('Amostra')

# Criação e exibição de tooltips no gráfico
mpld3.plugins.connect(fig, mpld3.plugins.MousePosition(fontsize=14))
mpld3.display()


Podemos definir um filtro, primariamente, através de:
* Resposta em frequência:
	* Filtro Passa-Baixas (Low-Pass Filter ou FPB), é um filtro que remove frequências acima da frequência de corte e mantém as frequências abaixo dela;
	* Filtro Passa-Altas (High-Pass Filter ou FPA), é um filtro que remove frequências abaixo da frequência de corte e mantém as frequências acima dela;
	* Filtro Passa-Faixa (Band-Pass Filter ou FPF), é um filtro que remove frequências fora de uma faixa definida pelas frequências de corte inferior e superior e mantém as frequências dentro dela;
* Filtro Rejeita-Faixa (Band-Stop Filter ou FRF), é um filtro que remove frequências dentro de uma faixa definida pelas frequências de corte inferior e superior e mantém as frequências fora dela.
* Resposta ao impulso:
	* Filtro de Resposta Finita ao Impulso (Finite Impulse Response ou FIR), são filtros cuja saída se torna nula após um tempo finito, quando excitado por um impulso. Outra forma de escrever isso é dizer que a resposta do filtro depende apenas das entradas dele (não possui realimentação);
	* Filtro de Resposta Infinita ao Impulso (Infinite Impulse Response ou IIR), são filtros cuja saída não se torna nula após um tempo finito, quando excitado por um impulso. Outra forma de escrever isso é dizer que a resposta do filtro depende das entradas e saídas passadas dele (possui realimentação).
* Ondulações na reposta em frequência:
	* Flat: Não possui ondulações na resposta em frequência;
	* Tipo I: Possui ondulações (ripple) na banda de passagem;
	* Tipo II: Possui ondulações (ripple) na banda de rejeição.
* Otimizado para resposta no tempo ou em frequência:
	* Tempo: Distorcem o menos o possível a resposta temporal, em detrimento da resposta em frequência. São ideia para sinais com informações no tempo, como sinais pulsados;
	* Frequência: Distorcem o menos possível a resposta em frequência, em detrimento da resposta temporal. São ideias para sinais com informações na frequência, como sinais de áudio.

A escolha do filtro vai depender muito do problema em que ele é aplicado, mas filtros FIR costumam a serem mais bem comportados e mais lentos, enquanto filtros IIR costumam a serem mais rápidos ao custo ondulações e deformações.


##Filtros de Resposta Finita ao Impulso (FIR):

Alguns dos exemplos mais comuns de filtros FIR são os filtros por Média Móvel, Sinc-Janelado e Filtro personalizado.

O filtro por Média Móvel (Moving Average) é um dos filtros mais comuns em PDS e é representado pela média simples das últimas $M$ amostras do sinal, sendo M o tamanho do filtro. Possui fase linear e suas principais vantagens são a alta facilidade de implementação e grande velocidade de execução dele. Esse filtro é otimizado para o domínio do tempo e é ótimo para a tarefa de reduzir ruído aleatório do sinal enquanto mantém uma resposta a degrau aguda. Por outro lado, esse é o pior filtro para o demônio da frequência, tendo baixa capacidade de separar diferentes bandas de frequência. Podemos especificar esse filtro em função do quão rápido queremos que o filtro demore para expressar uma resposta ao degrau, em número de amostras.

A resposta ao impulso desse tipo de filtro é simples o bastante: $N$ amostras seguidas de amplitude $1/M$. Podemos escrever esse filtro como:
> $y[i] = \frac{1}{M} \sum_{j=0}^{M-1}x[i+j]$ \\
$y[i] = \frac{1}{M} \sum_{j=-(M-1)/2}^{(M-1)/2}x[i+j]$ \\
$y[i] = \frac{1}{M} \sum_{j=0}^{M-1}x[i-j]$

In [20]:
# Criação de um filtro de média móvel e resposta ao impulso

# Caracteristicas dos filtros
filter_taps = 127

# Geração dos coeficientes do filtro de média móvel
filter_coefficients_moving_avg = np.ones(filter_taps)/filter_taps

# Geração de um impulso unitário e um degrau unitário
impulse_response_samples = 1024
signal_impulse = signal.unit_impulse(impulse_response_samples)

# Geração da resposta ao impulso dos filtros
impulse_response_moving_avg = signal.lfilter(filter_coefficients_moving_avg, 1.0, signal_impulse)
#impulse_response_moving_avg = signal.convolve(filter_coefficients_moving_avg, signal_impulse)

# Geração da resposta em frequência dos filtros
fft_filter_moving_avg = np.fft.fftshift(np.fft.fft(impulse_response_moving_avg, impulse_response_samples))
fft_freqs = np.fft.fftshift(np.fft.fftfreq(impulse_response_samples))

# Criação da figura
fig = plt.figure()

# Criação do primeiro subplot
plt.subplot(2, 2, 1)
plt.plot(fft_freqs, 20*np.log10(np.absolute(fft_filter_moving_avg)), "-b", label='Moving Average')
plt.legend()
plt.title('Magnitudes dos filtros de Média Móvel')
plt.ylabel('Ganho [dB]')
plt.xlabel('Frequência [ciclos por amostra]')

# Criação do segundo subplot
plt.subplot(2, 2, 2)
plt.plot(fft_freqs, np.absolute(fft_filter_moving_avg), "-b", label='Moving Average')
plt.legend()
plt.title('Magnitudes dos filtros de Média Móvel')
plt.ylabel('Ganho [-]')
plt.xlabel('Frequência [ciclos por amostra]')

# Criação do terceiro subplot
plt.subplot(2, 2, 3)
plt.plot(impulse_response_moving_avg[:filter_taps], "-b", label='Moving Average')
plt.legend()
plt.title('Resposta ao impulso unitário da Média Móvel')
plt.ylabel('Amplitude')
plt.xlabel('Amostra')

# Criação do quarto subplot
plt.subplot(2, 2, 4)
plt.plot(fft_freqs, np.angle(fft_filter_moving_avg), "-b", label='Moving Average')
plt.legend()
plt.title('Fases dos filtros de Média Móvel')
plt.ylabel('Fase [rad]')
plt.xlabel('Frequência [ciclos por amostra]')


# Criação e exibição de tooltips no gráfico
mpld3.plugins.connect(fig, mpld3.plugins.MousePosition(fontsize=14))
mpld3.display()

O filtro por Sinc-Janelado (Windowing-Sinc) são filtros altamente estáveis, possuem fase linear e podem atingir níveis excelentes de performance. São otimizados para respostas em frequência, tendo baixa performance no domínio do tempo, apresentando ondulações e overshoots excessivos em respostas ao degrau. 

Esses filtros usam a função $sinc(x)$ como base para tentar gerar filtros ideias em frequência. Como a representação temporal da função $sinc(x)$ é infinita e precisa ser truncada, a resposta em frequência sofre distorções significativas. Aplicando uma janela (como a de Hamming ou a de Blackman) nesse $sinc(x)$ truncado, podemos reduzir as distorções na resposta em frequência, chegando assim a um filtro de Sinc-Janelado.

Em particular, as janelas de Hamming e Blackman citadas acimas são comumente usadas por produzirem boas respostas. A janela de Hamming produz filtros com decaimento mais rápido do que as de Blackman (cerca de 20%), mas com piores oscilações nas bandas de passagem e de rejeição (0,02% para Blackman contra 0,2% para Hamming).


In [21]:
# Criação de um filtro de sinc-janelado e resposta ao impulso

# Caracteristicas dos filtros
filter_taps = 127
cutout_freq = 0.2
bandwidth_freq = np.array([0.15, 0.25])

# Geração dos coeficientes dos filtros com janela Hamming
#filter_coefficients_hamming = signal.firwin(filter_taps, bandwidth_freq*2, window='hamming', pass_zero='bandpass')
filter_coefficients_hamming = signal.firwin(filter_taps, cutout_freq*2, window='hamming', pass_zero='lowpass')
#filter_coefficients_hamming = signal.firwin(filter_taps, cutout_freq*2, window='hamming', pass_zero='highpass')
#filter_coefficients_hamming = signal.firwin(filter_taps, bandwidth_freq*2, window='hamming', pass_zero='bandstop')

# Geração dos coeficientes dos filtros com janela Blackman
#filter_coefficients_blackman = signal.firwin(filter_taps, bandwidth_freq*2, window='blackman', pass_zero='bandpass')
filter_coefficients_blackman = signal.firwin(filter_taps, cutout_freq*2, window='blackman', pass_zero='lowpass')
#filter_coefficients_blackman = signal.firwin(filter_taps, cutout_freq*2, window='blackman', pass_zero='highpass')
#filter_coefficients_blackman = signal.firwin(filter_taps, bandwidth_freq*2, window='blackman', pass_zero='bandstop')

# Geração de um impulso unitário
impulse_response_samples = 1024
signal_impulse = signal.unit_impulse(impulse_response_samples)

# Geração da resposta ao impulso dos filtros
impulse_response_hamming = signal.lfilter(filter_coefficients_hamming, 1.0, signal_impulse)
impulse_response_blackman = signal.lfilter(filter_coefficients_blackman, 1.0, signal_impulse)
#impulse_response_hamming = signal.convolve(filter_coefficients_hamming, signal_impulse)
#impulse_response_blackman = signal.convolve(filter_coefficients_blackman, signal_impulse)

# Geração da resposta em frequência dos filtros
fft_filter_hamming = np.fft.fftshift(np.fft.fft(impulse_response_hamming, impulse_response_samples))
fft_filter_blackman = np.fft.fftshift(np.fft.fft(impulse_response_blackman, impulse_response_samples))
fft_freqs = np.fft.fftshift(np.fft.fftfreq(impulse_response_samples))

# Criação da figura
fig = plt.figure()

# Criação do primeiro subplot
plt.subplot(2, 2, 1)
plt.plot(fft_freqs, 20*np.log10(np.absolute(fft_filter_hamming)), "-b", label='Hamming')
plt.plot(fft_freqs, 20*np.log10(np.absolute(fft_filter_blackman)), "--r", label='Blackman')
plt.legend()
plt.title('Magnitudes dos filtros de Sinc-Janelado')
plt.ylabel('Ganho [dB]')
plt.xlabel('Frequência [ciclos por amostra]')

# Criação do segundo subplot
plt.subplot(2, 2, 2)
plt.plot(fft_freqs, np.absolute(fft_filter_hamming), "-b", label='Hamming')
plt.plot(fft_freqs, np.absolute(fft_filter_blackman), "--r", label='Blackman')
plt.legend()
plt.title('Magnitudes dos filtros de Sinc-Janelado')
plt.ylabel('Ganho [-]')
plt.xlabel('Frequência [ciclos por amostra]')

# Criação do terceiro subplot
plt.subplot(2, 2, 3)
plt.plot(impulse_response_hamming[:filter_taps], "-b", label='Hamming')
plt.plot(impulse_response_blackman[:filter_taps], "--r", label='Blackman')
plt.legend()
plt.title('Resposta ao impulso unitário dos filtros de Sinc-Janelado')
plt.ylabel('Amplitude')
plt.xlabel('Amostra')

# Criação do quarto subplot
plt.subplot(2, 2, 4)
plt.plot(fft_freqs, np.angle(fft_filter_hamming), "-b", label='Hamming')
plt.plot(fft_freqs, np.angle(fft_filter_blackman), "--r", label='Blackman')
plt.legend()
plt.title('Fases dos filtros de Sinc-Janelado')
plt.ylabel('Fase [rad]')
plt.xlabel('Frequência [ciclos por amostra]')

# Criação e exibição de tooltips no gráfico
mpld3.plugins.connect(fig, mpld3.plugins.MousePosition(fontsize=14))
mpld3.display()

O filtro personalizado (Custom Filter) pode ser considerado o filtro mais adaptável possível, pois permite ao usuário especificar uma resposta em frequência completamente arbitraria. Ou seja, é possível conseguir qualquer ganho nas frequências individuais do sistema. Sua ideia é similar ao do filtro por Sinc-Janelado, mas em vez de se basear na função $sinc(x)$ ele usa métodos numéricos para gerar a resposta impulsiva do sistema. Para gerar um filtro personalizado, começamos pela sua resposta em frequência e vamos ao tempo:
* Inicialmente escolhemos a resposta em frequência desejada e geramos uma DFT com essas respostas;
* Então usamos métodos numéricos como a IFFT para gerar uma resposta impulsiva para a resposta em frequência desejada;
* Por último, truncamos o resultado com o tamanho de amostras do filtro, janelamos as amostras e criamos a resposta impulsiva do filtro a partir das amostras janeladas.

Uma observação importante é que quanto mais complexa a resposta em frequência, mais amostras no tempo serão necessárias. 


In [22]:
# Criação de um filtro personalizado e resposta ao impulso

# Resposta em frequência desejada
freq_samples = 1024
custom_freqs = np.linspace(0, 1, np.int(freq_samples/2))
custom_gains = np.linspace(1, 0, np.int(freq_samples/2))

# Caracteristicas dos filtros
filter_taps = 127

# Geração dos coeficientes do filtro personalizado com janela Hamming
fcustom_coefficients_hamming = signal.firwin2(filter_taps, custom_freqs, custom_gains, window='hamming')

# Geração dos coeficientes dos filtros com janela Blackman
fcustom_coefficients_blackman = signal.firwin2(filter_taps, custom_freqs, custom_gains, window='blackman')

# Geração de um impulso unitário
impulse_response_samples = 1024
signal_impulse = signal.unit_impulse(impulse_response_samples)

# Geração da resposta ao impulso dos filtros
impulse_response_hamming = signal.lfilter(fcustom_coefficients_hamming, 1.0, signal_impulse)
impulse_response_blackman = signal.lfilter(fcustom_coefficients_blackman, 1.0, signal_impulse)
#impulse_response_hamming = signal.convolve(fcustom_coefficients_hamming, signal_impulse)
#impulse_response_blackman = signal.convolve(fcustom_coefficients_blackman, signal_impulse)

# Geração da resposta em frequência dos filtros
fft_filter_hamming = np.fft.fftshift(np.fft.fft(impulse_response_hamming, impulse_response_samples))
fft_filter_blackman = np.fft.fftshift(np.fft.fft(impulse_response_blackman, impulse_response_samples))
fft_freqs = np.fft.fftshift(np.fft.fftfreq(impulse_response_samples))

# Criação da figura
fig = plt.figure()

# Criação do primeiro subplot
plt.subplot(2, 2, 1)
plt.plot(fft_freqs, 20*np.log10(np.absolute(fft_filter_hamming)), "-b", label='Hamming')
plt.plot(fft_freqs, 20*np.log10(np.absolute(fft_filter_blackman)), "--r", label='Blackman')
plt.legend()
plt.title('Magnitudes dos filtros de Sinc-Janelado')
plt.ylabel('Ganho [dB]')
plt.xlabel('Frequência [ciclos por amostra]')

# Criação do segundo subplot
plt.subplot(2, 2, 2)
plt.plot(fft_freqs, np.absolute(fft_filter_hamming), "-b", label='Hamming')
plt.plot(fft_freqs, np.absolute(fft_filter_blackman), "--r", label='Blackman')
plt.legend()
plt.title('Magnitudes dos filtros de Sinc-Janelado')
plt.ylabel('Ganho [-]')
plt.xlabel('Frequência [ciclos por amostra]')

# Criação do terceiro subplot
plt.subplot(2, 2, 3)
plt.plot(impulse_response_hamming[:filter_taps], "-b", label='Hamming')
plt.plot(impulse_response_blackman[:filter_taps], "--r", label='Blackman')
plt.legend()
plt.title('Resposta ao impulso unitário dos filtros de Sinc-Janelado')
plt.ylabel('Amplitude')
plt.xlabel('Amostra')

# Criação do quarto subplot
plt.subplot(2, 2, 4)
plt.plot(fft_freqs, np.angle(fft_filter_hamming), "-b", label='Hamming')
plt.plot(fft_freqs, np.angle(fft_filter_blackman), "--r", label='Blackman')
plt.legend()
plt.title('Fases dos filtros de Sinc-Janelado')
plt.ylabel('Fase [rad]')
plt.xlabel('Frequência [ciclos por amostra]')

# Criação e exibição de tooltips no gráfico
mpld3.plugins.connect(fig, mpld3.plugins.MousePosition(fontsize=14))
mpld3.display()

In [23]:
# Criação de um filtro de sinc-janelado e resposta ao impulso para filtrar a Tensão da Rede Elétrica

# Caracteristicas dos filtros
filter_taps = 127
cutout_freq = 120

# Geração dos coeficientes do filtro
filter_coefficients_voltage = signal.firwin(filter_taps, cutout_freq/sampling_rate_hz*2, window='hamming', pass_zero='lowpass')

# Geração de um impulso unitário
impulse_response_samples = 1024
signal_impulse = signal.unit_impulse(impulse_response_samples)

# Geração da resposta ao impulso dos filtros
impulse_response_voltage = signal.lfilter(filter_coefficients_voltage, 1.0, signal_impulse)
#impulse_response_voltage = signal.convolve(filter_coefficients_voltage, signal_impulse)

# Geração da resposta em frequência dos filtros
fft_filter_voltage = np.fft.fftshift(np.fft.fft(filter_coefficients_voltage, impulse_response_samples))
fft_freqs = np.fft.fftshift(np.fft.fftfreq(impulse_response_samples)) * sampling_rate_hz

# Criação da figura
fig = plt.figure()

# Criação do primeiro subplot
plt.subplot(2, 2, 1)
plt.plot(fft_freqs, 20*np.log10(np.absolute(fft_filter_voltage)))
plt.title('Magnitudes dos filtros de Sinc-Janelado')
plt.ylabel('Ganho [dB]')
plt.xlabel('Frequência [Hz]')

# Criação do segundo subplot
plt.subplot(2, 2, 2)
plt.plot(fft_freqs, np.absolute(fft_filter_voltage))
plt.title('Magnitudes dos filtros de Sinc-Janelado')
plt.ylabel('Ganho [-]')
plt.xlabel('Frequência [Hz]')

# Criação do terceiro subplot
plt.subplot(2, 2, 3)
plt.plot(impulse_response_voltage[:filter_taps])
plt.title('Resposta ao impulso unitário dos filtros de Sinc-Janelado')
plt.ylabel('Amplitude')
plt.xlabel('Amostra')

# Criação do quarto subplot
plt.subplot(2, 2, 4)
plt.plot(fft_freqs, np.angle(fft_filter_voltage))
plt.title('Fases dos filtros de Sinc-Janelado')
plt.ylabel('Fase [rad]')
plt.xlabel('Frequência [Hz]')

# Criação e exibição de tooltips no gráfico
mpld3.plugins.connect(fig, mpld3.plugins.MousePosition(fontsize=14))
mpld3.display()

In [25]:
# Aplicação de filtros na Tensão de uma Rede Elétrica Residencial

# Aplicação do filtro
quantized_voltage_samples_filtered = signal.lfilter(filter_coefficients_voltage, 1.0, quantized_voltage_samples)

# Criação da figura
fig = plt.figure()
#plt.plot(time_signal, voltage_signal, '-b')
#plt.plot(time_samples, quantized_voltage_samples, ',r')
plt.step(time_samples, quantized_voltage_samples_filtered, '-y')
plt.plot(time_samples, quantized_voltage_samples_filtered, 'dg')
plt.title('Tensão da Rede Eletrica Residencial, filtrada')
plt.ylabel('Amplitude [V]')
plt.xlabel('Tempo [s]')

# Criação e exibição de tooltips no gráfico
mpld3.plugins.connect(fig, mpld3.plugins.MousePosition(fontsize=14))
mpld3.display()

In [26]:
# Aplicação da FFT no sinal filtradado da Tensão de uma Rede Elétrica Residencial

# Aplicação da janela no sinal filtrado
quantized_voltage_samples_filtered_windowed = quantized_voltage_samples_filtered * signal.hann(quantized_voltage_samples_filtered.size)
#quantized_voltage_samples_filtered_windowed = quantized_voltage_samples_filtered * signal.hamming(quantized_voltage_samples_filtered.size)
#quantized_voltage_samples_filtered_windowed = quantized_voltage_samples_filtered * signal.blackman(quantized_voltage_samples_filtered.size)
#quantized_voltage_samples_filtered_windowed = quantized_voltage_samples_filtered * signal.flattop(quantized_voltage_samples_filtered.size)

# Execução do algoritmo da FFT
fft_windowed = np.fft.fft(quantized_voltage_samples_filtered_windowed, num_samples)

# Geração do vetor de frequências da FFT
freqs_windowed = np.fft.fftfreq(num_samples)

# Shift da FFT e do vetor de frequências
fft_windowed_shifted = np.fft.fftshift(fft_windowed)
freqs_windowed_shifted = np.fft.fftshift(freqs_windowed)

# Corrige o valor da FFT
fft_windowed_scaled = fft_windowed_shifted/num_samples
freqs_windowed_shifted_hz = freqs_windowed_shifted * sampling_rate_hz

# Seleciona a FFT a ser plotada
#fft_windowed_plot = fft_windowed
#fft_windowed_plot = fft_windowed_shifted
fft_windowed_plot = fft_windowed_scaled

# Seleciona o vetor de frequências ser plotado
#freqs_windowed_plot = freqs_windowed
#freqs_windowed_plot = freqs_windowed_shifted
freqs_windowed_plot = freqs_windowed_shifted_hz

# Criação da figura
fig = plt.figure()

# Criação do primeiro subplot
plt.subplot(2, 1, 1)
stem_fft_windowed_real = plt.stem(freqs_windowed_shifted_hz, np.absolute(fft_windowed_plot), use_line_collection=True)
stem_fft_windowed_real[0].set_markerfacecolor('none')
stem_fft_windowed_real[0].set_markersize(2)
plt.title('Magnitudes da FFT da Tensão Elétrica Residencial Filtrada')
plt.ylabel('Amplitude [V]')
plt.xlabel('Frequência [Hz]')

# Criação do segundo subplot
plt.subplot(2, 1, 2)
stem_fft_windowed_imag = plt.stem(freqs_windowed_shifted_hz, np.angle(fft_windowed_plot), use_line_collection=True)
stem_fft_windowed_imag[0].set_markerfacecolor('none')
stem_fft_windowed_imag[0].set_markersize(2)
plt.title('Fases da FFT da Tensão Elétrica Residencial Filtrada')
plt.ylabel('Fase [rad]')
plt.xlabel('Frequência [Hz]')

# Criação e exibição de tooltips no gráfico
mpld3.plugins.connect(fig, mpld3.plugins.MousePosition(fontsize=14))
labels_fft_windowed_real = ["Freq {:.3f} = {:.3f}".format(freqs_windowed_shifted_hz[i], np.absolute(fft_windowed_plot[i])) for i in range(freqs_windowed_shifted_hz.size)]
mpld3.plugins.connect(fig, mpld3.plugins.PointLabelTooltip(stem_fft_windowed_real[0], labels_fft_windowed_real))
labels_fft_windowed_imag = ["Freq {:.3f} = {:.3f}".format(freqs_windowed_shifted_hz[i], np.angle(fft_windowed_plot[i])) for i in range(freqs_windowed_shifted_hz.size)]
mpld3.plugins.connect(fig, mpld3.plugins.PointLabelTooltip(stem_fft_windowed_imag[0], labels_fft_windowed_imag))
mpld3.display()
